In [1]:
from selenium import webdriver
import pause
from bs4 import BeautifulSoup

In [87]:

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [100]:
from selenium.webdriver.support.ui import Select
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("--kiosk")

driver = webdriver.Chrome(options=options, executable_path=r'/Users/jbaripatti/Development/chromedriver')

PAGE = 'https://www.sephora.com/ca/en/shop/concealer?pageSize=300'

driver.get(PAGE)
pause.seconds(1)

#close pop-up
icon_cross_ele = driver.find_element_by_css_selector('svg.css-1bpyffv').click()
pause.seconds(2)




#select view all items
#icon_select_all = Select(driver.find_element_by_css_selector("select.css-bpk111"))
#icon_select_all.select_by_visible_text('View all')

pause.seconds(5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

pause.seconds(15)

# Wait 30 seconds for page to load and extract the element after it loads
#timeout = 30
#try:
    #WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "footer.css-2gkpw0")))
#except TimeoutException:
    #print('Timed out waiting for page to load')
    #driver.quit()
       

#footer = driver.find_element_by_tag_name('footer')

#footer.location_once_scrolled_into_view


page_source = driver.page_source
print(f"fetching {PAGE} from the internet")

soup = BeautifulSoup(page_source, 'html.parser')
driver.close()
driver.quit()

fetching https://www.sephora.com/ca/en/shop/concealer?pageSize=300 from the internet


In [101]:
# get all brands on the page, it results in a <class 'bs4.element.ResultSet'> iterable
brands = soup.find_all(attrs= {"data-at" : "sku_item_brand"})
type(brands)

bs4.element.ResultSet

In [102]:
for brand in brands:
    brand_name = brand.get_text()
    print(brand_name)

NARS
FENTY BEAUTY by Rihanna
Too Faced
FENTY BEAUTY by Rihanna
tarte
FENTY BEAUTY by Rihanna
NARS
IT Cosmetics
MAKE UP FOR EVER
Yves Saint Laurent
Urban Decay
BECCA
Bobbi Brown
BURBERRY
Estée Lauder
Lise Watier
LANEIGE
Urban Decay
Urban Decay
Urban Decay


In [103]:
item_grid = soup.find_all('div', class_= "css-dkxsdo")
grids = []
for grid in item_grid:
    
    grids.append(grid)
len(grids)

11

In [104]:
item_block = soup.find_all('div', class_="css-12egk0t")
rows = []
for item in item_block:
    rows.append(item)

In [105]:
len(rows)

128

In [106]:
# pasa algo despues del item 12
item_block = soup.find_all('div', class_="css-12egk0t")
for item in item_block:
    print(item.find(attrs={'data-at':'sku_item_brand'}).text)

NARS
FENTY BEAUTY by Rihanna
Too Faced
FENTY BEAUTY by Rihanna
tarte
FENTY BEAUTY by Rihanna
NARS
IT Cosmetics
MAKE UP FOR EVER
Yves Saint Laurent
Urban Decay
BECCA


AttributeError: 'NoneType' object has no attribute 'text'

In [107]:
print(item_block[:1])

[<div class="css-12egk0t"><a aria-label="NARS Radiant Creamy Concealer" class="css-ix8km1" data-comp="ProductItem" data-uid="P377873 2172310" href="/ca/en/product/radiant-creamy-concealer-P377873?icid2=products grid:p377873"><div class="css-79elbk"><div class="css-a8c8mi"><div class="css-j9nkok " data-comp="ProductImage Box"><div class="css-16bb1sx " data-comp="Box"></div><img alt="NARS - Radiant Creamy Concealer" class="css-1glglyy " data-comp="Box" src="/productimages/sku/s2172310-main-grid.jpg" srcset="/productimages/sku/s2172310-main-grid.jpg 1x, /productimages/sku/s2172310-main-grid@2x.jpg 2x"/></div><button aria-label="More info on NARS Radiant Creamy Concealer" class="css-1g0g0sz" data-comp="ProductQuicklook" type="button">Quick Look</button><div class="css-5emb8u" style="opacity:0"><button aria-label="Sign in to love NARS Radiant Creamy Concealer" class="css-q32zok" data-at="unloved" data-comp="ProductLoveToggle"><svg aria-hidden="true" class="css-18io71b " data-comp="IconLove 

In [108]:
print(item_block[12:13])

[<div class="css-12egk0t"><div class="css-13ys60n" data-comp="LazyLoad"></div></div>]


In [18]:
item = soup.find('div', class_="css-12egk0t")

In [23]:
brand = item.find(attrs={'data-at':'sku_item_brand'}).text
brand

'FENTY BEAUTY by Rihanna'

In [25]:
item_name = item.find(attrs={'data-at': 'sku_item_name'}).text
item_name

"Pro Filt'r Soft Matte Longwear Foundation"

In [26]:
item_price = item.find(attrs={'data-at': "sku_item_price_list" }).text
item_price

'C$45.00'

In [30]:
item_shades = item.find('div', class_="css-rrjz1n").text
item_shades

'49 more colors'

In [45]:
# get item rating, it's part of the attributes of the class, so I get the dictionary of attibutes
#and access the rating by key
item_rating = item.find('div', class_='css-1adflzz').attrs['aria-label']

'4 stars'

In [46]:
def extract_data(item):
    item_brand = item.find(attrs={'data-at':'sku_item_brand'}).text
    item_name = item.find(attrs={'data-at': 'sku_item_name'}).text
    item_price = item.find(attrs={'data-at': "sku_item_price_list" }).text
    item_shades = item.find('div', class_="css-rrjz1n").text
    item_rating = item.find('div', class_='css-1adflzz').attrs['aria-label']
    row = dict(brand=item_brand,
              name=item_name,
              price = item_price,
              n_shades=item_shades,
              rating=item_rating)
    return row

In [47]:
def process_item_block(soup):
    item_blocks = soup.find_all('div', class_="css-12egk0t")
    rows = []
    for item in item_blocks:
        row = extract_data(item)
        rows.append(row)
        
    return rows

In [50]:
process_item_block(soup)

AttributeError: 'NoneType' object has no attribute 'text'

In [55]:
soup

<html class="css-1esi5of" data-comp="Index" lang="en"><head class="at-element-marker" data-comp="Head"><script async="async" src='https:////tapestry.tapad.com/tapestry/1?ta_partner_id=1518&amp;rnd=9993076834949&amp;ta_analytics={"isNewSession":true}&amp;ta_set_local_var=s_3_Integrate_Tapad_get_0' type="text/javascript"></script><title>Foundation Makeup | Sephora</title><iframe src="javascript:void(0)" style="width: 0px; height: 0px; border: 0px; display: none;" title=""></iframe><script src="https://vt.myvisualiq.net/2/fCHI62undn4iESjl0SHs0A%3D%3D/vt-171.js" type="text/javascript"></script><script src="https://assistjs.skimresources.com/assist.js" type="text/javascript"></script><script src="https://s.btstatic.com/lib/1bb56fe346d9f2ccb47c6ef2d62eedb2e61d69b5.js?v=2" type="text/javascript"></script><script src="//s.thebrighttag.com/tag?site=eWeZ32Q&amp;H=-suxjbe&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fca%2Fen%2Fshop%2Ffoundation-makeup&amp;mode=v2&amp;cf=7273301" type="text/javascr